# Tutorial 3

**This tutorial will cover:**

* Dropping columns
* Dropping rows
* Parameters for the dropping functions
* Filling missing values

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [2]:
# Inspect the dataset.
df1 = spark.read.csv("test-data-3.csv", header=True, inferSchema=True)
df1.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Steve|  31|        10| 30000|
| Bill|  30|         8| 25000|
| John|  29|         4| 20000|
| Paul|  24|         3| 20000|
|Chris|  21|         1| 15000|
|  Tom|  23|         2| 18000|
|Frank|null|      null| 40000|
| null|  34|        10| 38000|
| null|  36|      null|  null|
+-----+----+----------+------+



## Dropping columns

In [3]:
df2 = spark.read.csv("test-data-3.csv", header=True, inferSchema=True)

In [4]:
# Dropping columns does NOT mutate the original dataframe. Instead it returns a new dataframe with the specified columns removed.
df2.drop("Name")

DataFrame[Age: int, Experience: int, Salary: int]

In [5]:
# So if you view the dataframe, you will still see the original dataframe.
df2.show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Steve|  31|        10| 30000|
| Bill|  30|         8| 25000|
| John|  29|         4| 20000|
| Paul|  24|         3| 20000|
|Chris|  21|         1| 15000|
|  Tom|  23|         2| 18000|
|Frank|null|      null| 40000|
| null|  34|        10| 38000|
| null|  36|      null|  null|
+-----+----+----------+------+



In [6]:
# In order to see the new dataframe you can save it to a variable and inspect it.
df2_dropped_col = df2.drop("Name")
df2_dropped_col.show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  31|        10| 30000|
|  30|         8| 25000|
|  29|         4| 20000|
|  24|         3| 20000|
|  21|         1| 15000|
|  23|         2| 18000|
|null|      null| 40000|
|  34|        10| 38000|
|  36|      null|  null|
+----+----------+------+



## Dropping rows

In [7]:
df3 = spark.read.csv("test-data-3.csv", header=True, inferSchema=True)

In [8]:
# Drop all rows that have at least 1 null value.
df3.na.drop().show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Steve| 31|        10| 30000|
| Bill| 30|         8| 25000|
| John| 29|         4| 20000|
| Paul| 24|         3| 20000|
|Chris| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



## Parameters for the dropping methods

In the previous code cell, if you right-click `drop()` and select "Show Contextual Help", then you will see that `drop()` accepts three parameters: `how`, `thresh` (i.e. threshhold), and `subset`. You can read the descriptions for how each of those parameters works. However, the `subset` parameter might be a little confusing. When you pass a list of column names to `subset`, then the setting that you passed to the `how` or `thresh` parameters will only be applied to the columns that you specified. For example:

In [9]:
df3.na.drop(how="any", subset=["Name"]).show()

+-----+----+----------+------+
| Name| Age|Experience|Salary|
+-----+----+----------+------+
|Steve|  31|        10| 30000|
| Bill|  30|         8| 25000|
| John|  29|         4| 20000|
| Paul|  24|         3| 20000|
|Chris|  21|         1| 15000|
|  Tom|  23|         2| 18000|
|Frank|null|      null| 40000|
+-----+----+----------+------+



## Filling missing values

You can use the `fill()` method to fill in missing values. The `fill()` method can take two parameters: `value` and `subset`. The `value` parameter is the value that will replace the `null` values. The `subset` parameter takes a list of column names that will be filled with the replacement values.

In [10]:
df4 = spark.read.csv("test-data-3.csv", header=True)

In [11]:
df4.na.fill("Missing Value").show()

+-------------+-------------+-------------+-------------+
|         Name|          Age|   Experience|       Salary|
+-------------+-------------+-------------+-------------+
|        Steve|           31|           10|        30000|
|         Bill|           30|            8|        25000|
|         John|           29|            4|        20000|
|         Paul|           24|            3|        20000|
|        Chris|           21|            1|        15000|
|          Tom|           23|            2|        18000|
|        Frank|Missing Value|Missing Value|        40000|
|Missing Value|           34|           10|        38000|
|Missing Value|           36|Missing Value|Missing Value|
+-------------+-------------+-------------+-------------+



If you look at the schema of the dataframe, then you will see that all of the column data types are `string`, which is why `Missing Value` replaced all the `null` values in the dataframe.

In [12]:
df4.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



However, if you pass `inferSchema=True` to the `csv()` method, then `Missing Value` will only fill in other values that have a string data type.

In [13]:
df5 = spark.read.csv("test-data-3.csv", header=True, inferSchema=True)
df5.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [14]:
df5.na.fill("Missing Value").show()

+-------------+----+----------+------+
|         Name| Age|Experience|Salary|
+-------------+----+----------+------+
|        Steve|  31|        10| 30000|
|         Bill|  30|         8| 25000|
|         John|  29|         4| 20000|
|         Paul|  24|         3| 20000|
|        Chris|  21|         1| 15000|
|          Tom|  23|         2| 18000|
|        Frank|null|      null| 40000|
|Missing Value|  34|        10| 38000|
|Missing Value|  36|      null|  null|
+-------------+----+----------+------+



You can replace null values in specific columns by passing a list of column names.

In [15]:
df4.na.fill("Missing Value", ["Age","Experience"]).show()

+-----+-------------+-------------+------+
| Name|          Age|   Experience|Salary|
+-----+-------------+-------------+------+
|Steve|           31|           10| 30000|
| Bill|           30|            8| 25000|
| John|           29|            4| 20000|
| Paul|           24|            3| 20000|
|Chris|           21|            1| 15000|
|  Tom|           23|            2| 18000|
|Frank|Missing Value|Missing Value| 40000|
| null|           34|           10| 38000|
| null|           36|Missing Value|  null|
+-----+-------------+-------------+------+



## Filling missing values with mean, median, or mode

In statistics, imputation is the process of replacing missing data with substituted values. ([Wikipedia](https://en.wikipedia.org/wiki/Imputation_(statistics)))

To get more details about the following example, you can read the PySpark docs for [Imputer](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Imputer.html).

In [16]:
# Specify the input columns and replace the missing values with the mean. The input columns should be of numeric type.
# The resulting dataframe will contain the original columns plus the output columns, which will contain the imputed values.

from pyspark.ml.feature import Imputer

# Setup the imputer configuration.
imputer = Imputer(
    inputCols=["Age", "Experience", "Salary"],
    outputCols=["{}_imputed".format(c) for c in ["Age", "Experience", "Salary"]]
).setStrategy("mean")

# You can also pass "median" or "mode" to the `setStrategy()` method.

In [17]:
# Add the imputation columns to the dataframe.
imputer.fit(df5).transform(df5).show()

+-----+----+----------+------+-----------+------------------+--------------+
| Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+------+-----------+------------------+--------------+
|Steve|  31|        10| 30000|         31|                10|         30000|
| Bill|  30|         8| 25000|         30|                 8|         25000|
| John|  29|         4| 20000|         29|                 4|         20000|
| Paul|  24|         3| 20000|         24|                 3|         20000|
|Chris|  21|         1| 15000|         21|                 1|         15000|
|  Tom|  23|         2| 18000|         23|                 2|         18000|
|Frank|null|      null| 40000|         28|                 5|         40000|
| null|  34|        10| 38000|         34|                10|         38000|
| null|  36|      null|  null|         36|                 5|         25750|
+-----+----+----------+------+-----------+------------------+--------------+